In [1]:
import pandas, re, warnings

warnings.filterwarnings("ignore")

In [2]:
names = ["MovieID", "Title", "Genre"]

origin = pandas.read_table("movies.csv", sep=",", names=names, skiprows=1)

origin.iloc[:5]

MovieID                               Title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                         Genre  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [3]:
origin.tail()

MovieID                                      Title  \
9737   193581  Black Butler: Book of the Atlantic (2017)   
9738   193583               No Game No Life: Zero (2017)   
9739   193585                               Flint (2017)   
9740   193587        Bungo Stray Dogs: Dead Apple (2018)   
9741   193609        Andrew Dice Clay: Dice Rules (1991)   

                                Genre  
9737  Action|Animation|Comedy|Fantasy  
9738         Animation|Comedy|Fantasy  
9739                            Drama  
9740                 Action|Animation  
9741                           Comedy

In [4]:
null = origin.isnull().sum()

null.sum()

np.int64(0)

In [5]:
total = origin.shape[0]

total

9742

In [6]:
total_film = origin["Title"].nunique()

total_film

9737

In [7]:
total == total_film

False

In [8]:
origin = origin.drop_duplicates("Title")

origin["Title"].nunique()

9737

In [9]:
assert origin.shape[0] == origin["Title"].nunique(), "Fail !"

In [10]:
splita = lambda val : " ".join(val.split("|"))

origin["Genre"] = origin["Genre"].apply(splita)

In [11]:
TotalGenre = []

for value in origin["Genre"].values:

  value = value.split(" ")

  for val in value:

    if val not in TotalGenre: TotalGenre.append(val)

len(TotalGenre)

22

In [12]:
TotalGenre[15:]

['Documentary', 'IMAX', 'Western', 'Film-Noir', '(no', 'genres', 'listed)']

In [13]:
nolist = "(no genres listed)"

origin = origin[origin["Genre"] != nolist]

origin.iloc[:5]

MovieID                               Title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                         Genre  
0  Adventure Animation Children Comedy Fantasy  
1                   Adventure Children Fantasy  
2                               Comedy Romance  
3                         Comedy Drama Romance  
4                                       Comedy

In [14]:
TotalGenre = []

for insect in origin["Genre"].values:

  insect = insect.split(" ")

  for val in insect:

    if val not in TotalGenre: TotalGenre.append(val)

len(TotalGenre)

19

In [15]:
replacer = {"Sci-Fi":"SciFi", "Film-Noir":"Filnoir"}

for i, t in replacer.items():

  origin["Genre"] = origin["Genre"].str.replace(i, t)

origin.iloc[:5]

MovieID                               Title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                         Genre  
0  Adventure Animation Children Comedy Fantasy  
1                   Adventure Children Fantasy  
2                               Comedy Romance  
3                         Comedy Drama Romance  
4                                       Comedy

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words="english")

metrics = tfidf.fit_transform(origin["Genre"])

metrics.shape

(9703, 19)

In [17]:
tfidf.get_feature_names_out().tolist()[:5]

['action', 'adventure', 'animation', 'children', 'comedy']

In [18]:
assert len(tfidf.get_feature_names_out().tolist()) == len(TotalGenre), "Fail !"

In [19]:
origin.loc[:1]

MovieID             Title                                        Genre
0        1  Toy Story (1995)  Adventure Animation Children Comedy Fantasy
1        2    Jumanji (1995)                   Adventure Children Fantasy

In [20]:
metrics.todense()[0]

matrix([[0.        , 0.41679332, 0.51629181, 0.50489821, 0.26739237,
         0.        , 0.        , 0.        , 0.48301679, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ]])

In [21]:
metrics.todense()[1]

matrix([[0.        , 0.51228317, 0.        , 0.62057343, 0.        ,
         0.        , 0.        , 0.        , 0.59367884, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ]])

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(metrics)

similarities.shape

(9703, 9703)

In [23]:
sample = similarities[0]

sample.shape

(9703,)

In [24]:
intler = lambda i : origin.iloc[i, 1]

titint = lambda t : origin[origin["Title"] == t].index.values[0]

In [25]:
titint("Jumanji (1995)")

np.int64(1)

In [26]:
title = "Jumanji (1995)"

similar_result = enumerate(similarities[int(titint(title))])

orilist = list(similar_result)

`1995 Jumanji Cosine Similarity`

In [27]:
title = "Jumanji (1995)"

similar_result = enumerate(similarities[int(titint(title))])

orilist = list(similar_result)

In [28]:
orilist = sorted(orilist, key=lambda i : i[1], reverse=True)

[(i, int(round(t, 3))) for i, t in orilist[:5]]

[(1, 1), (53, 1), (109, 1), (767, 1), (1514, 1)]

In [29]:
results = [intler(i) for i, _ in orilist[:5]]

origin[origin['Title'].isin(results)]

MovieID                                      Title  \
1           2                             Jumanji (1995)   
53         60         Indian in the Cupboard, The (1995)   
109       126          NeverEnding Story III, The (1994)   
767      1009            Escape to Witch Mountain (1975)   
1514     2043  Darby O'Gill and the Little People (1959)   

                           Genre  
1     Adventure Children Fantasy  
53    Adventure Children Fantasy  
109   Adventure Children Fantasy  
767   Adventure Children Fantasy  
1514  Adventure Children Fantasy

In [30]:
def logits(title, limit):

  target = list(enumerate(similarities[int(titint(title))]))

  target_sort = list(sorted(target, key=lambda i:i[1], reverse=True))

  logits_list = list(target_sort)

  result = list(filter(lambda i : i[0] != titint(title), logits_list[:limit + 1]))

  titles = [origin.iloc[i[0], 1] for i in result]

  return origin[origin["Title"].isin(titles)].iloc[:limit]

logits(title, 5)

MovieID                                      Title  \
53         60         Indian in the Cupboard, The (1995)   
109       126          NeverEnding Story III, The (1994)   
767      1009            Escape to Witch Mountain (1975)   
1514     2043  Darby O'Gill and the Little People (1959)   
1556     2093                        Return to Oz (1985)   

                           Genre  
53    Adventure Children Fantasy  
109   Adventure Children Fantasy  
767   Adventure Children Fantasy  
1514  Adventure Children Fantasy  
1556  Adventure Children Fantasy

In [31]:
title = origin.iloc[3574, 1]

logits(title, 10)

MovieID                                              Title  \
1           2                                     Jumanji (1995)   
53         60                 Indian in the Cupboard, The (1995)   
109       126                  NeverEnding Story III, The (1994)   
767      1009                    Escape to Witch Mountain (1975)   
1514     2043          Darby O'Gill and the Little People (1959)   
1556     2093                                Return to Oz (1985)   
1617     2161                      NeverEnding Story, The (1984)   
1618     2162  NeverEnding Story II: The Next Chapter, The (1...   
1799     2399                      Santa Claus: The Movie (1985)   
6075    41566  Chronicles of Narnia: The Lion, the Witch and ...   

                           Genre  
1     Adventure Children Fantasy  
53    Adventure Children Fantasy  
109   Adventure Children Fantasy  
767   Adventure Children Fantasy  
1514  Adventure Children Fantasy  
1556  Adventure Children Fantasy  
1617  Adventure Children Fantasy  
1618  Adventure Children Fantasy  
1799  Adventure Children Fantasy  
6075  Adventure Children Fantasy

`Cold Start Problem`

In [32]:
option = ["Children", "Fantasy"]

option

['Children', 'Fantasy']

In [33]:
def starter(option, limit):

  result = []

  unique_genres = origin["Genre"].unique()

  for item in unique_genres:

    spliter = item.split(" ")

    overlap = len(set(option) & set(spliter)) / len(spliter)

    if overlap > 0.5:

      result.append((item, overlap))

  result = sorted(result, key=lambda i:i[1], reverse=True)[:limit]

  result = [i[0] for i in result]

  result = origin[origin["Genre"].isin(result)].iloc[:limit]

  return result

In [34]:
starter(option, 10)

MovieID                               Title                       Genre
1          2                      Jumanji (1995)  Adventure Children Fantasy
53        60  Indian in the Cupboard, The (1995)  Adventure Children Fantasy
109      126   NeverEnding Story III, The (1994)  Adventure Children Fantasy
209      243                        Gordy (1995)     Children Comedy Fantasy
301      343       Baby-Sitters Club, The (1995)                    Children
313      355             Flintstones, The (1994)     Children Comedy Fantasy
355      410         Addams Family Values (1993)     Children Comedy Fantasy
637      810                       Kazaam (1996)     Children Comedy Fantasy
649      837                      Matilda (1996)     Children Comedy Fantasy
671      885                        Bogus (1996)      Children Drama Fantasy

In [35]:
title = "Gordy (1995)"

logits(title, 5)

MovieID                        Title                    Genre
313       355      Flintstones, The (1994)  Children Comedy Fantasy
355       410  Addams Family Values (1993)  Children Comedy Fantasy
637       810                Kazaam (1996)  Children Comedy Fantasy
649       837               Matilda (1996)  Children Comedy Fantasy
1282     1702               Flubber (1997)  Children Comedy Fantasy